### **title of whatever we're doing**

Amia Yi, Kevin Fan, Vincent ???

**Introduction**
- why are we analyzing this data and stuff
- introduce tutorial and what its for and why its useful
- look at the examples on the project desc for reference

In [2]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
df = pd.read_csv("C:\\Users\\aegir\\Documents\\dATASCI\\CMSC320\\Final Project\\Data\\3680211.csv")
df.sort_values("DATE", inplace=True)
# TODO: Figure out how to deal with data for the same year.
df.loc[df["DATE"]==1949]

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
50,USC00181995,"COLLEGE PARK, MD US",38.98333,-76.9500,27.4,1949,NaN,NaN,NaN,NaN,...,1033.8,",0",342.0,",0",NaN,NaN,20.7,0.0,NaN,NaN
173,USC00180705,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.9500,29.9,1949,578.8,NaN,578.8,0.0,...,984.9,",0",166.0,",0",12.8,0.0,19.4,0.0,6.3,0.0
203,USC00180706,"BELTSVILLE PLANT STATION, MD US",39.01667,-76.9500,67.1,1949,630.0,NaN,630.0,0.0,...,988.0,",0",166.0,",0",13.4,0.0,19.4,0.0,7.5,0.0
99,USC00180700,"BELTSVILLE, MD US",39.03020,-76.9315,44.2,1949,612.0,NaN,612.0,0.0,...,983.6,",0",NaN,NaN,12.9,0.0,19.1,0.0,6.7,0.0
